# Pruebas RAG

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

### 1️⃣ Preparar la Base de Datos de Sesgos
Este paso convierte frases con sesgos en vectores y las almacena en FAISS.

In [66]:
# Cargar modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Base de datos estructurada por categorías (5 ejemplos por cada categoría)
bias_examples = {
    "lenguaje_sexista": [
        "Las mujeres no sirven para la política.",
        "Los hombres no deben cocinar, eso es cosa de mujeres.",
        "Las chicas no entienden de ciencia.",
        "Los hombres que lloran son débiles.",
        "Las mujeres hablan demasiado y no escuchan."
    ],
    "hombre_humanidad": [
        "El hombre ha conquistado el espacio exterior.",
        "La historia del hombre es fascinante.",
        "El hombre moderno vive más años.",
        "El progreso del hombre se debe a su inteligencia.",
        "El hombre debe cuidar más del planeta."
    ],
    "dual_aparente": [
        "Los alumnos y las alumnas están invitados al evento.",
        "Todos los ciudadanos y las ciudadanas deben votar.",
        "Convocamos a profesores y profesoras a la reunión.",
        "Se espera la presencia de doctores y doctoras.",
        "Los niños y las niñas necesitan espacio para jugar."
    ],
    "cargos_mujeres": [
        "La presidenta mujer tomó una decisión rápida.",
        "La jueza mujer dictó sentencia.",
        "La médica mujer fue reconocida por su trabajo.",
        "La ingeniera mujer lidera el proyecto.",
        "La piloto mujer aterrizó el avión sin problemas."
    ],
    "sexismo_social": [
        "Detrás de cada hombre exitoso hay una mujer.",
        "Las mujeres deben ser delicadas y sumisas.",
        "Los niños juegan fútbol, las niñas hacen ballet.",
        "Las mujeres están más capacitadas para cuidar a los niños.",
        "Los hombres están hechos para liderar."
    ],
    "androcentrismo": [
        "Los grandes inventores, como Edison y Tesla, transformaron la sociedad.",
        "Los exploradores que descubrieron continentes eran hombres valientes.",
        "El progreso científico fue logrado por grandes hombres.",
        "La civilización fue construida por hombres fuertes y decididos.",
        "Los héroes de guerra son recordados por sus hazañas."
    ],
    "asimetria": [
        "Pedro Gómez y su esposa fueron a la ceremonia.",
        "El doctor García y su secretaria participaron en la conferencia.",
        "El empresario asistió acompañado por su asistente femenina.",
        "Luis y su novia asistieron al evento.",
        "El ingeniero presentó su proyecto junto a su ayudante mujer."
    ],
    "infantilizacion": [
        "La chica encargada de marketing hizo un buen trabajo.",
        "Esa niña del departamento de ventas tiene potencial.",
        "La muchacha que lleva la contabilidad es eficiente.",
        "La jovencita que contrataron demuestra iniciativa.",
        "La chica nueva se está adaptando rápido al equipo."
    ],
    "denominacion_sexualizada": [
        "La nueva secretaria es muy atractiva.",
        "La doctora que llegó es una belleza.",
        "Contrataron una camarera hermosa para atraer clientes.",
        "La nueva presentadora tiene un cuerpo impresionante.",
        "La recepcionista, muy guapa, atiende muy bien."
    ],
    "denominacion_redundante": [
        "La mujer policía resolvió rápidamente el caso.",
        "La mujer soldado demostró valentía.",
        "La mujer árbitro tomó una buena decisión.",
        "La mujer bombero salvó varias vidas.",
        "La mujer juez dictaminó correctamente."
    ],
    "denominacion_dependiente": [
        "Laura, la esposa del ministro, inauguró el evento.",
        "Carmen, hija del empresario, ganó el premio.",
        "Marta, la hermana del futbolista, fue entrevistada.",
        "Ana, esposa del presidente, asistirá al acto.",
        "Lucía, la novia del actor famoso, lanzó su marca."
    ]
}


# Convertir la lista de ejemplos en embeddings
all_examples = [example for examples in bias_examples.values() for example in examples]
bias_embeddings = np.array([embedding_model.encode(text) for text in all_examples])

# Crear índice FAISS y añadir embeddings
index = faiss.IndexFlatL2(bias_embeddings.shape[1])
index.add(bias_embeddings)

### 2️⃣ Buscar Sesgos en un Texto Nuevo
Cuando entra un texto, buscamos frases similares en la base de datos.

In [67]:
def retrieve_relevant_examples(query, k=2):
    """ Busca en FAISS los ejemplos más similares al texto dado y devuelve ejemplos con sus categorías """
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    _, idxs = index.search(query_embedding, k)

    # Crear una lista plana de ejemplos con sus respectivas categorías
    all_examples_with_categories = [
        (example, category)
        for category, examples in bias_examples.items()
        for example in examples
    ]

    # Recuperar ejemplos relevantes con su categoría asociada
    relevant_examples = [all_examples_with_categories[i] for i in idxs[0]]

    return relevant_examples

In [68]:
# Texto a analizar
new_text = "Las jugadoras chicas han realizado un gran partido."

In [69]:
# Recuperar ejemplos similares desde FAISS
retrieved_examples = retrieve_relevant_examples(new_text)
for example, category in retrieved_examples:
    print(f"Categoría: {category}, Ejemplo: '{example}'")

Categoría: lenguaje_sexista, Ejemplo: 'Las chicas no entienden de ciencia.'
Categoría: cargos_mujeres, Ejemplo: 'La jueza mujer dictó sentencia.'


### 3️⃣ Generar el Prompt y Pasarlo a LLM
Integramos la información recuperada en un prompt y pedimos a un LLM que analice el texto.

#### Salamandra

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Cargar Salamandra-2B
model_name = "BSC-LT/salamandra-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [38]:
# Construir el prompt con los ejemplos recuperados
prompt = f"""Eres un experto en lenguaje inclusivo y sesgos de género. Analiza el siguiente texto:

Ejemplos de sesgos detectados:
{retrieved_examples}

Clasifica el texto a analizar según los siguientes criterios (0 = No, 1 = Sí, 2 = Sí con salto semántico). Devuelve la respuesta en el siguiente formato:

Ejemplo de respuesta:
Lenguaje sexista: 1 (La frase implica que solo los hombres han conquistado el espacio, excluyendo a las mujeres).
Uso de 'hombre' para denominar a la humanidad: 1 (Se usa 'hombre' para referirse a la humanidad en general, en lugar de términos neutros como 'humanidad').
Uso dual aparente: 0
Uso de cargos/profesiones para mujeres: 0
Sexismo social: 0
Androcentrismo: 1 (Se centra en los logros masculinos sin mencionar el papel de las mujeres).
Asimetría en tratamiento: 0
Infantilización de mujeres: 0
Denominación sexualizada: 0
Denominación redundante: 0
Denominación dependiente: 0

Ahora, analiza el siguiente texto y responde siguiendo el formato anterior:

Texto a analizar: "{new_text}"

Respuesta:"""

In [40]:
# Tokenizar e inferir
inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    output = model.generate(**inputs,
                            max_new_tokens=200,
                            do_sample=True,
                            # temperature=0.1
                            )

# Decodificar la respuesta
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("\nRespuesta de Salamandra:")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Respuesta de Salamandra:
Eres un experto en lenguaje inclusivo y sesgos de género. Analiza el siguiente texto:

Ejemplos de sesgos detectados:
['El hombre promedio es más racional que una mujer.', 'Las mujeres son emocionales y no aptas para liderar.']

Clasifica el texto a analizar según los siguientes criterios (0 = No, 1 = Sí, 2 = Sí con salto semántico). Devuelve la respuesta en el siguiente formato:

Ejemplo de respuesta:
Lenguaje sexista: 1 (La frase implica que solo los hombres han conquistado el espacio, excluyendo a las mujeres).
Uso de 'hombre' para denominar a la humanidad: 1 (Se usa 'hombre' para referirse a la humanidad en general, en lugar de términos neutros como 'humanidad').
Uso dual aparente: 0
Uso de cargos/profesiones para mujeres: 0
Sexismo social: 0
Androcentrismo: 1 (Se centra en los logros masculinos sin mencionar el papel de las mujeres).
Asimetría en tratamiento: 0
Infantilización de mujeres: 0
Denominación sexualizada: 0
Denominación redundante: 0
Denominaci

#### OpenAI

In [46]:
import configparser
import openai
from pydantic import BaseModel
import json
from tqdm import tqdm
import pandas as pd

In [75]:
class LenguajeSexista(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class HombreHumanidad(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class DualAparente(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class CargosMujeres(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class SexismoSocial(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class Androcentrismo(BaseModel):
    etiqueta: int
    explicación: str
    ejemplos_articulo: list[str]

class Asimetria(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class Infantilizacion(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class DenominacionSexualizada(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class DenominacionRedundante(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class DenominacionDependiente(BaseModel):
    etiqueta: int
    explicacion: str
    ejemplos_articulo: list[str]

class ResponseFormat(BaseModel):
    lenguaje_sexista: LenguajeSexista
    hombre_humanidad: HombreHumanidad
    dual_aparente: DualAparente
    cargos_mujeres: CargosMujeres
    sexismo_social: SexismoSocial
    androcentrismo: Androcentrismo
    asimetria: Asimetria
    infantilizacion: Infantilizacion
    denominacion_sexualizada: DenominacionSexualizada
    denominacion_redundante: DenominacionRedundante
    denominacion_dependiente: DenominacionDependiente

config = configparser.ConfigParser()
config.read('../pruebas_openai/config.ini')

OPENAI_MODEL = config['DEFAULT']['OPENAI_MODEL']
openai.api_key = config['DEFAULT']['OPENAI_API_KEY']

In [56]:
data = pd.read_csv("../data/contenido_2023_04_05_matriz_not_julia.csv")
data = data.iloc[[9, 10]]

In [57]:
data

,Unnamed: 0,N,n_periodico,enlace,Medio,año,Fecha,Nombreperiodista,Nombreperiodista_A,Género_periodista,...,@25ge3ero1oto,@25perspectiva,@25piedefoto,@26ge3ero1oto,@26perspectiva,@26piedefoto,@27ge3ero1oto,@27perspectiva,@27piedefoto,contenido
9,9,129,129,https://www.infolibre.es/cultura/esther-vivas-...,1,2019,20/3/2019,Clara Morales,Clara Morales,1,...,,,,,,,,,,"Dice Esther Vivas (Sabadell, 1975) que ""la lit..."
10,10,133,133,https://www.infolibre.es/politica/cientificas-...,1,2019,6/2/2019,Europa Press,Agencia /otros medios,4,...,,,,,,,,,,Científicas han debatido este miércoles sobre ...


In [81]:
predictions = []

# Loop de predicción
for idx, row in tqdm(data.iterrows(), total=len(data), desc="Procesando predicciones"):
    n = row['N']
    n_periodico = row['n_periodico']
    contenido = row['contenido']
    try:
        # Llamada al modelo para detectar las citas humanas y sus fuentes
        completion = openai.beta.chat.completions.parse(
            model=OPENAI_MODEL,
            messages=[
                {
                    "role": "system", 
                    "content": """
                                    Eres un especialista en análisis de lenguaje inclusivo y detección de sesgos de género en textos, especialmente en artículos.

                                    Tu tarea consiste en analizar artículos e identificar distintos tipos de sesgos de género y lenguaje sexista según las siguientes categorías predefinidas:

                                    categorias = {
                                        "lenguaje_sexista": "Expresiones que excluyen o discriminan explícitamente un género.",
                                        "hombre_humanidad": "Uso de 'hombre' para referirse al conjunto de la humanidad en lugar de un término inclusivo.",
                                        "dual_aparente": "Uso innecesario del masculino y femenino cuando no es relevante semánticamente.",
                                        "cargos_mujeres": "Referencia específica o innecesaria al género femenino en profesiones o cargos.",
                                        "sexismo_social": "Expresiones que reflejan prejuicios o estereotipos sociales sobre género.",
                                        "androcentrismo": "Enfoque en la perspectiva o experiencia masculina excluyendo a otros géneros.",
                                        "asimetria": "Diferente tratamiento lingüístico o semántico injustificado hacia mujeres u hombres.",
                                        "infantilizacion": "Expresiones que presentan a las mujeres de manera infantil o inmadura.",
                                        "denominacion_sexualizada": "Uso de términos que sexualizan innecesariamente a las personas, generalmente mujeres.",
                                        "denominacion_redundante": "Referencia redundante o innecesaria al género.",
                                        "denominacion_dependiente": "Definición o descripción de una mujer en términos de un hombre u otra persona."
                                    }

                                    Importante:
                                    - Si el artículo menciona o discute sesgos, estereotipos o lenguaje sexista con una intención crítica, de análisis feminista o para visibilizarlos, esto NO debe considerarse como un sesgo del propio artículo. En esos casos, indica etiqueta = 0 y explícala claramente.

                                    Para cada categoría debes proporcionar una etiqueta, una explicación breve y una lista de ejemplos textuales del artículo usando este formato de respuesta:

                                    respuesta = {
                                        "categoria": {
                                            "etiqueta": 0,  # 0: No | 1: Sí | 2: Sí con salto semántico
                                            "explicacion": "breve explicación de por qué se asignó esta etiqueta.",
                                            "ejemplos_articulo": "lista de frases literales del artículo que evidencien esta categoría."
                                        },
                                        # repetir para cada categoría...
                                    }

                                    Asegúrate siempre de que:
                                    - Tu análisis sea objetivo, claro y conciso.
                                    - La explicación esté justificada por ejemplos específicos del texto.
                                    - Se sigan estrictamente las categorías y formato indicados anteriormente.
                                """
                },
                {
                    "role": "user", 
                    "content": f"""
                                    Clasifica el texto a analizar según los siguientes criterios (0 = No, 1 = Sí, 2 = Sí con salto semántico). Devuelve la respuesta en el siguiente formato, asegurándote de incluir tanto la etiqueta como una breve explicación para cada categoría:

                                    Categorías:
                                    - lenguaje_sexista: Uso explícito o implícito de lenguaje discriminatorio basado en el sexo o género.
                                    - hombre_humanidad: Uso de la palabra "hombre" para referirse al conjunto de la humanidad, ignorando alternativas neutras como "humanidad".
                                    - dual_aparente: Uso innecesario o inapropiado del masculino y femenino cuando una forma neutra sería suficiente.
                                    - cargos_mujeres: Utilización específica del género femenino en cargos o profesiones destacando innecesariamente el género.
                                    - sexismo_social: Reflejo de actitudes o estereotipos sexistas socialmente arraigados.
                                    - androcentrismo: El texto pone énfasis en la perspectiva masculina, ignorando o minimizando la femenina.
                                    - asimetria: Tratamiento desigual o asimétrico entre géneros en situaciones equivalentes.
                                    - infantilizacion: Trato hacia las mujeres como seres dependientes o infantiles.
                                    - denominacion_sexualizada: Descripción o denominación que resalta aspectos sexuales de manera inapropiada.
                                    - denominacion_redundante: Uso innecesario del género cuando la denominación ya es clara y neutra.
                                    - denominacion_dependiente: Descripción de una mujer únicamente por su relación o dependencia respecto a un hombre.

                                    
                                    Ahora, analiza el siguiente texto y responde siguiendo el formato requerido:

                                    Texto a analizar: "{contenido}"
                                    """
                                                }
            ],
            response_format=ResponseFormat
        )
        
        # Extraer la respuesta
        response_message = completion.choices[0].message.content
        print(response_message)

        # Usamos json.loads para convertir el string JSON a un objeto Python
        response_data = json.loads(response_message)

        # Agregar el índice de la noticia al JSON
        response_data['indice'] = idx  # Agrega el índice al diccionario de cada cita
        response_data['N'] = n
        response_data['n_periodico'] = n_periodico
        
        # Agregar el resultado con el índice a las predicciones
        predictions.append(response_data)

    except Exception as e:
        print(f"Error en el índice {idx}: {e}")

Procesando predicciones:   0%|          | 0/2 [00:00<?, ?it/s]

Procesando predicciones:  50%|█████     | 1/2 [00:09<00:09,  9.16s/it]

{
  "lenguaje_sexista": {
    "etiqueta": 0,
    "explicacion": "El texto no utiliza lenguaje sexista y aborda diversos aspectos de la maternidad de manera crítica y analítica.",
    "ejemplos_articulo": []
  },
  "hombre_humanidad": {
    "etiqueta": 0,
    "explicacion": "No se utiliza 'hombre' para referirse al conjunto de la humanidad, ya que se emplean términos más inclusivos.",
    "ejemplos_articulo": []
  },
  "dual_aparente": {
    "etiqueta": 0,
    "explicacion": "No se observa un uso innecesario del masculino y femenino; el texto es coherente y utiliza términos adecuados.",
    "ejemplos_articulo": []
  },
  "cargos_mujeres": {
    "etiqueta": 0,
    "explicacion": "Si bien se menciona la autoría de una mujer, esto es relevante y contextual, no se hace referencia innecesaria a su género en su rol.",
    "ejemplos_articulo": []
  },
  "sexismo_social": {
    "etiqueta": 0,
    "explicacion": "El texto critica expresamente las actitudes sexistas existentes y la violencia obst

Procesando predicciones: 100%|██████████| 2/2 [00:20<00:00, 10.18s/it]

{
  "lenguaje_sexista": {
    "etiqueta": 0,
    "explicacion": "El artículo aborda las desigualdades de género en la ciencia y propone cambios sociales sin emplear lenguaje sexista.",
    "ejemplos_articulo": []
  },
  "hombre_humanidad": {
    "etiqueta": 0,
    "explicacion": "No se utiliza 'hombre' como sinónimo de humanidad, se hace referencia a 'la mujer' de forma explícita y correcta.",
    "ejemplos_articulo": []
  },
  "dual_aparente": {
    "etiqueta": 0,
    "explicacion": "No se observan usos innecesarios del masculino y femenino, el texto usa términos adecuados al contexto.",
    "ejemplos_articulo": []
  },
  "cargos_mujeres": {
    "etiqueta": 1,
    "explicacion": "El artículo menciona específicamente a las mujeres en la ciencia, destacando su género en contextos relevantes para la discusión.",
    "ejemplos_articulo": [
      "han alertado de la desigualdad femenina en este campo",
      "la visibilización del talento femenino",
      "ha abogado porque se reviertan es

In [82]:
# Guardar las predicciones en un archivo JSON
with open('predictions_lenguaje.json', 'w', encoding='utf-8') as f:
    json.dump(predictions, f, ensure_ascii=False, indent=4)